In [72]:
import pandas as pd

In [73]:
data = pd.read_csv('./to_process/Sales.csv')

FileNotFoundError: [Errno 2] No such file or directory: './to_process/Sales.csv'

In [ ]:
data

,order_id,product_id,cost_per_unit,sales_per_unit,unit_sold
0,1,1,2,5,15
1,2,2,3,6,10


In [ ]:
data.dtypes

order_id          int64
product_id        int64
cost_per_unit     int64
sales_per_unit    int64
unit_sold         int64
dtype: object

Fix types

In [ ]:
data["cost_per_unit"] = data["cost_per_unit"].astype(float)
data["sales_per_unit"] = data["sales_per_unit"].astype(float)

Check and drop duplicates

In [ ]:
data = data.drop_duplicates()

Check cost_per_unit , sales_per_unit and unit_sold are not negative, if yes drop them

In [ ]:
data = data[data["cost_per_unit"] >= 0]

In [ ]:
data = data[data["sales_per_unit"] >= 0]

In [ ]:
data = data[data["unit_sold"] >= 0]

Calculate Total revenue

In [ ]:
data["revenue"] = (data["sales_per_unit"] - data["cost_per_unit"]) * data["unit_sold"]

Load to DB

connect to db (transaction pooler)

In [ ]:
from sqlalchemy import create_engine
from sqlalchemy.engine import URL
import os

connection_url = URL.create(
    "postgresql+psycopg2",
    username="postgres.belrumdjnpwghdykdtob",
    password= os.getenv("DB_PASSWORD"),
    host="aws-0-eu-central-2.pooler.supabase.com",
    port=6543,
    database="postgres",
)

engine = create_engine(connection_url)

conn = engine.connect()
conn.close()


OperationalError: (psycopg2.OperationalError) connection to server at "aws-0-eu-central-2.pooler.supabase.com" (51.96.34.188), port 6543 failed: fe_sendauth: no password supplied

(Background on this error at: https://sqlalche.me/e/20/e3q8)

In [ ]:
data

,order_id,product_id,cost_per_unit,sales_per_unit,unit_sold,revenue
0,1,1,2.0,5.0,15,45.0
1,2,2,3.0,6.0,10,30.0


Load the data to table

In [ ]:
data = data[["product_id","cost_per_unit","sales_per_unit","unit_sold","revenue"]]

In [ ]:
data

,product_id,cost_per_unit,sales_per_unit,unit_sold,revenue
0,1,2.0,5.0,15,45.0
1,2,3.0,6.0,10,30.0


In [ ]:
data.to_sql("orders",engine, if_exists="append", index=False)

2

move .csv to processed

In [ ]:
import shutil
shutil.move("to_process/Sales.csv","processed/Sales.csv")

'processed/Sales.csv'